In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from tkinter import Entry
from tkinter import Label
from tkinter import Button

# Функция для анализа данных и создания Excel-файла
def analyze_data():
    try:
        file_name = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        # Значение количества прикладываний от пользователя
        count_threshold = int(count_entry.get())
        df = pd.read_csv(file_name, sep=';')
        df.columns = ["Дата и время поездки", "", "Дата и время авторизации", "", "ID-терминала", "Номер маршрута", "ID-владельца маршрута", "Владелец маршрута", "Номер ТС", "ID-перевозчика", "Владелец ТС", "Категория платежа", "Тип транзакции", "", "Пересадка", "Сумма", "Идентификатор БК или ТК"]
        # Создаем объект ExcelWriter для записи в файл Excel
        save_file_name = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
        writer = pd.ExcelWriter(save_file_name, engine='xlsxwriter')
        # Анализ школьных карт
        school_df = df[df['Категория платежа'] == "Школьный проездной"]
        process_data(school_df, count_threshold, writer, 'Школьные карты')
        social_df = df[df['Категория платежа'] == "СТК"]
        process_data(social_df, count_threshold, writer, 'Социальные карты')
        citizen_df = df[(df['Категория платежа'] == "Проездной") & (df['Сумма'] == 0.0)]
        process_data(citizen_df, count_threshold, writer, 'Карта горожанина') 
        writer.save()
        messagebox.showinfo("Успех", f"Результаты сохранены в {save_file_name}")
    except Exception as e:
        messagebox.showerror("Ошибка", f"Произошла ошибка: {str(e)}")

# Функция для обработки данных и сохранения их в Excel
def process_data(data, count_threshold, writer, sheet_name):
# Группируем данные по TRM_ID и CARD
    data["Дата и время поездки"] = data['Дата и время поездки'].astype(str) +" "+ data.iloc[:, 1].astype(str)
    data["Дата и время авторизации"] = data['Дата и время авторизации'].astype(str) +" "+ data.iloc[:, 3].astype(str)
    grouped = data.groupby(['ID-терминала', 'Идентификатор БК или ТК'])
    data['count_trm'] = grouped['Идентификатор БК или ТК'].transform('count')
    data = data[data['count_trm'] >= count_threshold] # Фильтр по числу прикладывани
    #data["Дата и время поездки"] = data["Дата и время поездки"].str + " " + data["Дата и время авторизации"]
    data['LAST_4_TIMES'] = grouped['Дата и время поездки'].transform(lambda x: '/'.join(x.tail(4)))
    #data = data[['Дата и время поездки', 'ID-терминала', 'Идентификатор БК или ТК', 'count_trm', 'LAST_4_TIMES']] # Добавляем столбец CAT
    data = data[["Дата и время поездки", "Дата и время авторизации", "Идентификатор БК или ТК", "ID-терминала", "Номер маршрута", "Номер ТС", "ID-перевозчика", "Категория платежа", "Тип транзакции", "count_trm"]]
    data = data.drop_duplicates(subset=["Идентификатор БК или ТК"])
    data['Дата и время поездки'] = pd.to_datetime(data['Дата и время поездки'], format='%d.%m.%Y %H:%M:%S').dt.strftime('%Y-%m-%d')
    data.to_excel(writer, sheet_name=sheet_name, index=False)

root = tk.Tk()
root.title("Кража... показать ссылку")
root.geometry("400x200")

count_label = Label(root, text="Количество прикладываний:")
count_label.pack()
count_entry = Entry(root, width=10)
count_entry.pack()

analyze_button = Button(root, text="Поиск", command=analyze_data)
analyze_button.pack()

root.mainloop()

<ipython-input-1-bb291b266ec7>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Дата и время поездки"] = data['Дата и время поездки'].astype(str) +" "+ data.iloc[:, 1].astype(str)
<ipython-input-1-bb291b266ec7>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Дата и время авторизации"] = data['Дата и время авторизации'].astype(str) +" "+ data.iloc[:, 3].astype(str)
<ipython-input-1-bb291b266ec7>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try